In [48]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# Utility

In [97]:
# get race information
def get_race_info(soup):

    result = soup.find('div', class_ = 'f_fs13')
    lines = list(result.stripped_strings)

    # find venue
    venue = lines[1]
    venue = venue.split(',')
    venue = venue[3]
    venue = venue.strip()
    if venue == 'Sha Tin':
        venue = 'ST'
    elif venue == 'Happy Valley':
        venue = 'HV'
    else:
        venue = 'not found'

    # find track, course, and distance
    info = lines[2].split(',')

    track = info[0].strip()
    course = info[1].replace('"', '').replace('Course', '').strip()
    distance = info[2].strip()

    # find class
    race_class = lines[3].split(',')
    race_class = race_class[3].replace('Class', '').strip()

    return venue, track, course, distance, race_class

In [104]:
# get all horse href for characteristic
def get_race_horse(soup):
    all_links = []

    # find the table containing the horses url
    parti_horses = soup.find('table', class_ = 'starter f_tac f_fs13 draggable hiddenable')

    if not parti_horses:
        print('horses table not found')

    header_row = parti_horses.find('tr')

    headers = [td.get_text(strip = True) for td in header_row.find_all('td')]
    if 'Horse' not in headers:
        print('no horse column found')

    # get the column index of the horse column
    horse_col_index = headers.index('Horse')

    for row in parti_horses.find_all('tr')[1:]:
        cells = row.find_all('td') # cell in the row
        if len(cells) > horse_col_index:
            td = cells[horse_col_index]
            a_tag = td.find('a', href = True)
            if a_tag:
                href = a_tag['href']
                if href.startswith('/'):
                    href = 'https://racing.hkjc.com' + href
                all_links.append(href)
    
    return all_links


# Main

In [ ]:
def scrape_current_race(url):
    http_headers = {"User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"}
    response = requests.get('https://racing.hkjc.com/racing/information/English/Racing/Entries.aspx?RaceDate=2025/09/07&Racecourse=ST&View=Section&SectionNo=1&ProvRaceNo=P10', headers = http_headers)

    if response.status_code != 200:
        print(f'races not found')
    else:
        print('url found')

    soup = BeautifulSoup(response.text, 'html.parser')

    # from soup, find the race specific information
    venue, track, course, distance, race_class = get_race_info(soup)

    # get the horse url for scraping horse related information
    paricipating_horse_links = get_race_horse(soup)

In [100]:
http_headers = {"User-Agent": "Mozilla/5.0 (compatible; HKJCScraper/1.0)"}
response = requests.get('https://racing.hkjc.com/racing/information/English/Racing/RaceCard.aspx?RaceDate=2025/09/07&Racecourse=ST&RaceNo=1', headers = http_headers)

if response.status_code != 200:
    print(f'races not found')
else:
    print('url found')

url found


In [101]:
soup = BeautifulSoup(response.text, 'html.parser')

In [102]:
venue, track, course, distance, race_class = get_race_info(soup)

In [103]:
print(venue, track, course, distance, race_class)

ST Turf A 1200M 5


In [105]:
parti_links = get_race_horse(soup)

In [106]:
parti_links

['https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2023_J471',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2019_D090',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2024_K052',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2024_K028',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2023_J171',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2023_J441',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2024_K115',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2023_J065',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2022_H108',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?HorseId=HK_2022_H465',
 'https://racing.hkjc.com/racing/information/english/Horse/Horse.aspx?

In [55]:
result = soup.find('div', class_ = 'f_fs13')

In [57]:
lines = list(result.stripped_strings)

In [58]:
lines

['Race 1 - GRASSY HILL HANDICAP',
 'Sunday, September 07, 2025, Sha Tin, 13:00',
 'Turf, "A" Course, 1200M',
 'Prize Money: $875,000, Rating: 40-0, Class 5']

In [67]:
# find venue
venue = lines[1]
venue = venue.split(',')
venue = venue[3]
venue = venue.strip()
if venue == 'Sha Tin':
    venue = 'ST'
elif venue == 'Happy Valley':
    venue = 'HV'
else:
    venue = 'not found'


In [76]:
# find track, course, and distance
info = lines[2].split(',')

track = info[0].strip()
course = info[1].replace('"', '').replace('Course', '').strip()
length = info[2].strip()

Turf A 1200M


In [81]:
# find class
race_class = lines[3].split(',')
race_class = race_class[3].replace('Class', '').strip()
race_class

'5'

In [92]:
all_links = []

# find the table containing the horses url
parti_horses = soup.find('table', class_ = 'starter f_tac f_fs13 draggable hiddenable')

if not parti_horses:
    print('horses table not found')

header_row = parti_horses.find('tr')

In [95]:
headers = [td.get_text(strip = True) for td in header_row.find_all('td')]
if 'Horse' not in headers:
    print('no horse column found')

# get the column index of the horse column
horse_col_index = headers.index('Horse')

for row in parti_horses.find_all('tr')[1:]:
    cells = row.find_all('td') # cell in the row
    if len(cells) > horse_col_index:
        td = cells[horse_col_index]
        a_tag = td.find('a', href = True)
        if a_tag:
            href = a_tag['href']
            if href.startswith('/'):
                href = 'https://racing.hkjc.com' + href
            all_links.append(href)
            

In [96]:
all_links

['https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J471',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2019_D090',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K052',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K028',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J171',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J441',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2024_K115',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2023_J065',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H108',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?HorseId=HK_2022_H465',
 'https://racing.hkjc.com/racing/information/English/Horse/Horse.aspx?